<h1 style="color:#1398A1">TRAINING<h1>


<h3 style="color:#CCC229">LIBRAIRIES IMPORT<h3>


In [6]:
import sys
from pathlib import Path

src_path = Path("..")
sys.path.append(str(src_path))
%load_ext autoreload
%autoreload 2

from app.src.agent.utils.infos import list_snake_models
from app.src.agent.training.train import train_snake

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<h3 style="color:#CCC229">SOME TESTS<h3>

In [7]:
# Example of listing all available Snake models
list_snake_models()

🔍 Recherche des modèles dans snakeRL/snake-rl-models...

             MODELE (UUID)                   ||                 DESCRIPTION
--------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------
✅ Nombre de modèles trouvés : 0


In [9]:
# Example of training a Snake agent using PPO algorithm
train_snake(timesteps=1_000_000, grid_size=5, n_envs=5, algorithm="PPO", game_mode="walls")

2026/01/26 23:46:12 INFO mlflow.store.db.utils: Creating initial MLflow database tables...


✨ Nouvel agent : Grille 5x5, Mode walls, 5 envs.


2026/01/26 23:46:13 INFO mlflow.store.db.utils: Updating database tables
2026/01/26 23:46:13 INFO alembic.runtime.migration: Context impl PostgresqlImpl.
2026/01/26 23:46:13 INFO alembic.runtime.migration: Will assume transactional DDL.
2026/01/26 23:46:13 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
2026/01/26 23:46:13 INFO alembic.runtime.migration: Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
2026/01/26 23:46:14 INFO alembic.runtime.migration: Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
2026/01/26 23:46:14 INFO alembic.runtime.migration: Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
2026/01/26 23:46:15 INFO alembic.runtime.migration: Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
2026/01/26 23:46:15 INFO alembic.runtime.migration: Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
2026/01/26 23:46:15 INFO 


Démarrage Run MLflow : NEW_TRAINING_20260126_234612_c8dc9b36
Using cpu device
Go pour 1000000 steps...
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 9.11     |
|    ep_rew_mean     | -0.869   |
| time/              |          |
|    fps             | 9799     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 10240    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 8.11        |
|    ep_rew_mean          | -0.839      |
| time/                   |             |
|    fps                  | 4316        |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 20480       |
| train/                  |             |
|    approx_kl            | 0.009758607 |
|    clip_fraction        | 0.0872      |
|    clip_range           | 0.2         |
|    entro

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

2026/01/26 23:59:24 WARNING mlflow.pyfunc: Failed to infer model signature: Type hint <input: <class 'numpy.ndarray'>, output: None> cannot be used to infer model signature and input example is not provided, model signature cannot be inferred.
2026/01/26 23:59:25 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/26 23:59:25 INFO mlflow.store.db.utils: Updating database tables
2026/01/26 23:59:26 INFO alembic.runtime.migration: Context impl PostgresqlImpl.
2026/01/26 23:59:26 INFO alembic.runtime.migration: Will assume transactional DDL.
Successfully registered model 'Snake_5x5'.
Created version '1' of model 'Snake_5x5'.


Terminé ! Modèle walls dispo : c8dc9b36-8449-4394-a403-7c4417ddcb0e


In [ ]:
# Example of resuming training from a previous model using its UUID
train_snake(timesteps=1_000_000, algorithm="PPO", base_uuid="5cc88d67-1e48-40ae-943b-6528780da426")